$$
\newcommand{\fudm}[2]{\frac{\mathrm{D} #1}{\mathrm{D} #2}}
\newcommand{\pad}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\ppad}[2]{\frac{\partial^2 #1}{\partial #2^2}}
\newcommand{\ppadd}[3]{\frac{\partial^2 #1}{\partial #2 \partial #3}}
\newcommand{\nnabla}{\nabla^2}
\newcommand{\eps}{\epsilon}
\newcommand{\vdetail}[1]{\vb{#1}=\begin{pmatrix}#1_1\\#1_2\\#1_3\end{pmatrix}}
\newcommand{\vb}[1]{\mathbf{#1}}
\newcommand{\va}[1]{\vec{#1}}
\newcommand{\tb}[1]{\underline{\underline{\mathbf{#1}}}}
\newcommand{\fud}[2]{\frac{\mathrm{d} #1}{\mathrm{d} #2}}
$$

# Interface tracking

We can also represent the interface with a continous line and advect the interface according to the local flow velocity. To do so we need first a data structure to represent the interface, which is rater simple in 2d, yet 3d flow needs to take care much more. The 1d interface for 2d flows is stored as a 1d array

$$x_f(l)=(x(l),y(l))\quad l=1,2,..., N_f\quad.\tag{1}$$

The distance between points $l-1$ and $l$ can be easily calculated by the norm

$$\Delta s_{l,l-1}=\sqrt{\left(x(l)-x(l-1)\right)^2+\left(y(l)-y(l-1)\right)^2}\quad .\tag{2}$$

For convenience the array has two more points, $x_f+1$ and $x_f+2$ which are identical to $l=1$ and $l=2$. A sketch of an interface is given in [Fig. 1](#fig1).

<a name="fig1"></a>
<img src="../pics/interface rep.png" width=450> <p>
<center> Figure 1 </center>

## Moving the Front
The points on the front will move with an interpolated velocity of a first nearby grid point. We therefore need to identify the gridpoint which is clearly different for $u$ and $v$ and for the vertical and horizontal direction. Let's start for the horizontal direction, the $u$ velocity is available at center of the grid point and we obtain the index $i$ by the following operation

<a name="eq3"></a>$$i=\mathrm{FLOOR}(x_f(l)\,/\,\Delta x)+1\tag{3}$$

The operation $\mathrm{FLOOR}$ obtains the integer value of a floating point value, e.g. 

In [1]:
import math
xf=2.7
dx=1.
math.floor(xf/dx)+1

3.0

Thus [Eq. (3)](#eq3) gives the closeby grid point for the $u$ velocity in $i$-direction. For the vertical velocity which is defined only at positions $i+1/2$ we need to add this $1/2\,\Delta x$, thus $i$ is obtained by

<a name="eq4"></a>$$i=\mathrm{FLOOR}\left((x_f(l)+0.5\,\Delta x)\,/\,\Delta x\right)+1\quad .\tag{4}$$


In [2]:
xf=2.7
dx=1.
math.floor((xf+0.5*dx)/dx)+1

4.0

## Bilinear interpolation
Both operation are demonstrated displayed in [Fig. 2](#fig2). $i$ for the horizontal velocity starts with 1 with the velocity on the wall, while for the vertical velocity it starts with the ghost cell. The $\mathrm{floor}$ command generates a grid cell closeby and takes care that $x_f$ is in the interval $i \le x_f \le i+1$. The same holds for the vertical velocity and the pressure, yet [Eq. (4)](#eq4) has to be used.

<a name="fig2"></a>
<img src="../pics/front tracking_gridding.png" width=450> <p>
<center> Figure 2.</center>


To obtain the velocity at position x_f, we can use a linear interpolation:

$$u(x_f)=u_i \frac{x_{i+1}-x_f}{\Delta x} + u_{i+1} \frac{x_f-x_i}{\Delta x}\tag{5}$$

Please check that Eq. (5) gives the right result for $x_f=x_i$ and $x_f=x_{i+1}$, i.e. $u(x_i)$ and $u(x_{i+1})$, respectively.

If an arbitrary scalar function $\phi$, such as $u$, $v$, or $p$ is a dependent of two variables, $x_i$ and $y_j$, then one can introduce a bilinear approximation of the kind:

\begin{eqnarray}
\phi(x_f,y_f)=&\phi&_{i,j}\left(\frac{x_{i+1}-x_f}{\Delta x}\right)\left(\frac{y_{j+1}-y_f}{\Delta y}\right)+
\phi_{i,j+1}\left(\frac{x_{i+1}-x_f}{\Delta x}\right)\left(\frac{y_f-y_j}{\Delta y}\right)+\tag{6}\\
&\phi&_{i+1,j}\left(\frac{x_f-x_{i}}{\Delta x}\right)\left(\frac{y_{j+1}-y_f}{\Delta y}\right)+
\phi_{i+1,j+1}\left(\frac{x_f-x_i}{\Delta x}\right)\left(\frac{y_f-y_j}{\Delta y}\right)
\end{eqnarray}

Equation (3.6) can be rewritten as 

$$\phi(x_f,y_f)=w_{i,j}\phi_{i,j}+w_{i,j+1}\phi_{i,j+1}+w_{i+1,j}\phi_{i+1,j}+w_{i+1,j+1}\phi_{i+1,j+1}\tag{7}$$

where $w_{i,j}$ can be interpreted as weights. This nonlinear interpolation can is more clearly represented in Fig. 3 where the yellow point is multiplied with the yellow area, the blue point with the blue area and so on.

<a name="fig3"></a>
<img src="../pics/Bilinear_interpolation_visualisation.png" width=250> <p>
<center> Figure 3, taken from [Wikipedia](https://en.wikipedia.org/wiki/Bilinear_interpolation).</center>

The sum of the weights must obay

$$\sum_{i,j}w_{i,j}=1\quad .\tag{8}$$

We will use the bilinear interpolation to obtain the velocity at the interface. This will done separately for the $u$ and the $v$-component. Thereafter we can propagate the position of the front as

$$\vb{x}^{n+1}_f=\vb{x}^n_f+\Delta t \,\vb{u}^n_f\quad .\tag{9}$$


## Density reconstruction

The idea is to calculate first the gradient of the density from the tracked interface, and then integrate this gradient to obtain the density. This density field is updated and used for the next time step of the VoF calculation. 

Thus we need first to obtain this gradient which is done by defining the density with the help of Heavyside functions $H(\vb{x})$, which is 1 if $\vb{x}$ is in fluid 1 with density $\rho_1$ and 0 if $\vb{x}$ is in fluid 2 with density $\rho_2$.
Then the density in the whole fluid domain can be written as 

$$\rho(\vb{x})=\rho_1 \, H(\vb{x}) + \rho_2\,\left(1-H(\vb{x})\right)\quad .\tag{10}$$

The gradient can be applied in a straightforward fashion

$$\nabla \rho=(\rho_1-\rho_2)\,\nabla H(\vb{x})=\Delta \rho \,\vb{n} \,\delta (\vb{x}-\vb{x}_f)\quad . \tag{11}$$

The normal vector $\vb{n}$ points from fluid 2 to fluid 1 (why?) and the delta function $\delta$ is 1 only on the interface, i.e. $\vb{x}-\vb{x}_f=0$. 

We have now to connect the interface value (per area or in 2d per line element) with the value on the grid (per volume). This needs not only some kind of smoothing and but also the understanding how both are related.

<a name="fig4"></a>
<img src="../pics/interface integration.png" width=250> <p>
<center> Figure 4, taken from Tryggvason tutorial.</center>

Figure 4 shows a domain/volume where the interface cuts through. We can integrate the gradient of the density over this volume and use Gauss to write it as three surface integrals:

\begin{eqnarray}
\int_V \nabla \rho \, dV&=&\oint_S \rho \,\vb{n}\, dS\\
&=&\rho_1 \oint_{S_1} \vb{n}\, dS + \rho_2 \oint_{S_2} \vb{n}\, dS + \oint_{S_I} \rho \, \vb{n}\, dS\tag{12}\\
&=&\int_{S_I} (\rho_1 - \rho_2)\,\vb{n}\, ds\quad .
\end{eqnarray}

where $S_I$ is a surface element of the interface (a line element for 2d volumes). The closed surface integrals in Eq. (12) over $S_1$ and $S_2$ are 0. We can now write a general equation relating the per volume quantity $\nabla \rho$ with the per surface quantity $\Delta \rho\,\vb{n}$:

$$\int_V \nabla \rho \, dV=\int_{S_I} \Delta \rho\,\vb{n}\, ds\quad .\tag{13}$$

### Calculation of the gradient
In the program the gradient is distributed over the for closest cells using the bilinear interpolation. Thus we need 4 equations for the gradient in $x$ and 4 equations for the gradient in $y$-direction.


<a name="fig5"></a>
<img src="../pics/density_gradient_ft.png" width=500> <p>
<center> Figure 5</center>


Using Eq. (13) we can develop write down the density gradients for all 8 point. Figure 5 shows the position of the gradients in $x$-direction. As an example using the values $a_x$ and $a_y$ as the distances to the right side in $x$ and $y$-direction we can write for the upper left gradient:

$$\pad{\rho}{x}_{i-1/2,j}=\frac{\Delta \rho}{2} \frac{1}{\Delta x\, \Delta y}
\left(y_f(l+1)-y_f(l-1)\right)\,a_x\,a_y\tag{14}$$

This is done for all points of the front. As a grid may be visited multiple times we need to increase the value of the gradient at each grid point by the amount. 

### Integration
After the gradients in $x$ and $y$-direction are calculated, they can be integrated and to obtain the density field for the next time step. This is rather straightforward, we start with a point at some distance form the front where we know the density and integrate the density along one direction, e.g.

$$\rho_{i,j}=\rho_{i-1,j}+\Delta x \left(\pad{\rho}{x}\right)_{i-1/2,j}\tag{15}$$

The accuracy can be enhanced by integrating for point $(i,j)$ along all 4 directions and average the density. This results to 

$$\rho_{i,j}=\frac{1}{4}\left[\rho_{i-1,j}+\rho_{i+1,j}+\rho_{i,j-1}+\rho_{i,j+1}+\\
\Delta x \left(\left(\pad{\rho}{x}\right)_{i-1/2,j} - \left(\pad{\rho}{x}\right)_{i+1/2,j} \right)+
\Delta y \left(\left(\pad{\rho}{x}\right)_{i,j-1/2} - \left(\pad{\rho}{x}\right)_{i,j+1/2} \right)\right]\tag{16}$$

Equation (16) can be iteratively solved or to speed up we can use SOR. Note, the negative signs in Eq. (16) are from the direction of integration, if we integrate in positive $x$ and $y$-direction we have a positive $\Delta x$ while in the other directions $\Delta x$ and $\Delta y$ is negative.


## Reconstruction of the front


When the front is advected it may happen that points become closer and or they separate. To keep a fidel front for the furture advection steps we need to do some housekeeping. There we either remove points which are too close and add in points where distances become too large. Figure 6 depicts the linear list of points and the changes to the list during restructuring.
<a name="fig6"></a>
<img src="../pics/front restructuring.png" width=280> <p>
<center> Figure 6, taken from Tryggvason tutorial</center>

In the code this can be implemented by starting with a copy of the front points, deleting the original points, and then copy back the points where the check on distance is done point by point. If the distance is above a minimum distance and below a maximum distance, the point is copied.
If the distance is too large a point is inserted by placing it at half the distance. If the point are too close, it is not copied back into the list. To close the list of points the last point is copied to the first position of the list (this even works when the last points is deleted).
Inserting of points can be done simply by a linear interpolation, yet more careful insertion using high order interpolation increases stabilty and resolution.
